<a href="https://www.kaggle.com/code/ayushs9020/re-creating-protein-neural-nets-from-scratch?scriptVersionId=136117704" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Protein Neural Network

$Protein$ $Neural$ $Network$ is a type of `deep learning neural network` that is used to `predict` the `properties of proteins`. They are `trained` on a `large dataset` of `protein sequences`/`structures`. The network learns to `identify patterns` in the data that are associated with `particular properties`. $Protein$ $Neural$ $Networks$ have been shown to be `very effective` at `predicting protein properties`. In many cases, they `outperform traditional` methods that rely on `hand-crafted features`

<img src = "https://scontent.fjai2-1.fna.fbcdn.net/v/t1.6435-9/37298434_654990401529724_543197132539035648_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=730e14&_nc_ohc=7hJcasqo1PkAX8fs55i&_nc_ht=scontent.fjai2-1.fna&oh=00_AfBLiYMGoJgSMX0w-kd690Y2GvRX4ga07ER2OXTSeH7o5w&oe=64CE544A" width = 400>


# 1 | Basic Terminologies 💻

* $Structure$ $of$ $a$ $Protein$
* $Gene$ $Ontology$ $(GO)$
* $Principal$ $Component$ $Analaysis$
* $Transformers$
* $Categorical$ $Cross$ $Entropy$
* $Adaptive$ $Moment$ $Estimation$ $ADAM$

## 1.1 | Structure of a Protein

So what is a actually a **Protein...?**

First of all lets understand the structure of an **Atom**

<img src = "https://www.sciencefacts.net/wp-content/uploads/2020/11/Parts-of-an-Atom-Diagram.jpg"  width = 300>

There is a really good image I found of the `structure of atom`. Though there are many debates on the structure like this, but this `model is accepted universaly at this moment`.

In the centre we have the `Neucleus`. The `Neucleus` is made up of $2$ more structures named as `Neutron` and `Proton`. A `Proton` is `positively charged element` and a `Neutron` is a `neutral charged element`. A `Electron`, `negatively charged element`, `orbits` this `Neucleus` at some `distance apart`.

The `more we increase the number` of `Electrons` and `Protons`. The `bigger the atoms becomes`.

There are `different shells` where the `Electrons reside`. The `more closer the shell` is, the `less Electrons` it contrains. There are mainly $4$ shells. 

|||
|---|---
|$K$|$2$
|$L$|$8$
|$M$|$18$
|$N$|$32$

Once an atom `fills its outer most shell` with `Electrons`. It becomes `stable atom` and try to `refuse any donation` or `recieve of extra atom`.

<img src = "https://cdn1.byjus.com/wp-content/uploads/2022/01/word-image128.png" width = 400>

`Different atoms combine` to `share Electrons` and become `stable Molecules` 

<img src = "https://www.astrochem.org/sci_img/Amino_Acid_Structure.jpg" width = 300>

A `Amino Acid` is made up of mainly $4$ different atoms
`[H , C , O , N]`. 

<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/L-amino_acid_structure.svg/1200px-L-amino_acid_structure.svg.png" alt = "Bro use Light Theme" width = 300 >

We also have a free `Electron Pair` of `Carbon` in this molecule, we call this as a `Side Chain` which can be of different types. Basically this `Side Chain` provide the flexibility to make `different types` of `Amino Acids`. This flexibilty allows for $20$ `different` `Amino Acids` 

When we join `Amino Acids` with `peptide bonds`, we get `Proteins`. Conncecting different types of `Amino Acids` ends up in different types of `Proteins`.

## 1.2 | Gene Ontology (GO)

$Gene$ $Ontology$ $(GO)$ is a `controlled vocabulary` that `describes the functions of genes` and gene products. It is constantly being updated as new information becomes available.

There are mainly $3$ `Ontologies`

||||
|---|---|---
|$Biological$ $Process$|Describes the `biological processes`|A gene product might be involved in the process of `cell cycle`/`signal transduction.`
|$Cellular$ $Component$|Describes the `cellular components`|A gene product might be located in the `nucleus`/`cytoplasm`.
|$Molecular$ $Function$|Describes the `molecular functions`|A gene product might be involved in the `catalysis of a reaction`/`binding of a molecule`.

**[Gene Ontology Documentation](http://geneontology.org/docs/ontology-documentation/)**

## 1.3 | Principal Component Analysis

<img src = "https://media.makeameme.org/created/pca-the-cause.jpg" width = 400>

$Principal$ $Component$ $Analysis$ $PCA$ is a `statistical procedure` that uses an `orthogonal transformation` to convert a set of `correlated variables` into a set of `uncorrelated variables` called principal components.

First we compute the Covariance Matrix with the formula 

$$Cov_{x , y} = \frac{\sum(x_i - x_{mean})(y_i - y_{mean})}{N-1}$$

Then we find the Eign Values and Eign Vectors 

## 1.4 | Transformers 

A $Transformer$ is a `deep learning architecture` that relies on the `attention mechanism`. It is notable for requiring `less training time`. The model takes in `tokenized` `byte pair encoding` `input` tokens, and at each layer, `contextualizes each token` with other (unmasked) input tokens in `parallel` via attention mechanism.

* $Attention$ $Mechanism$- The attention mechanism allows the transformer to `learn long-range dependencies` between `tokens` in a sequence. 
* $Self-Attention$ - Self-attention is a special type of attention that allows the transformer to `attend to itself`. This means that the transformer can learn relationships between different parts of the same sequence.
* $Encoder-Decoder$ $Architecture$ - The transformer is an `Encoder-Decoder` architecture. This means that the transformer has two parts
* * Encoder - takes the input sequence and produces a sequence of hidden states
* * Decoder - takes these hidden states and produces the output sequence.

$$a = Softmax(\frac{KQ^T}{\sqrt{d_k}})$$

<img src = "https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png" width = 400>

## 1.5 | Categorical Cross Entropy 

$Loss$ is a `measure` of `how well a model is performing` on a given task. It is `calculated` by `comparing` the `model's predictions` to the `ground truth labels`. The `lower the loss`, the `better the model` is performing.

Here we will be using `Cross Entropy Loss` , a loss function that `measures` the `difference` between the `model's predicted probability distribution` and the `ground truth distribution`.

## 1.6 | Adaptive Moment Estimation 

An $Optimizer$ is an `algorithm` or function that `updates the weights and biases` of a neural network in order to `minimize a loss function`. 

Here we will be using the `Adam Optimizer`

$Adam$ is an $Adaptive$ $Learning$ $Rate$ method, which works by `maintaining two moving averages of the gradients`
* $Mean$ - Calculate the `momentum term`, which helps to `prevent` the `optimizer` from `getting stuck in local minima`
* $Variance$ - Calculate the `learning rate`, which is `adjusted based on the magnitude of the gradients`.

$$m_t = \beta_1 * m_{t - 1} + (1 - \beta_1) * w_t$$

$$v_t = \beta_2 * m_{t - 1} + (1 - \beta_2) * w_t$$

$$m_t = \frac{m_t}{1 - \beta_1^t}$$

$$v_t = \frac{v_t}{1 - \beta_2^t}$$

$$w_{t+1} = w_t - \frac{n}{\sqrt{v_t + e}} * m_t$$

# 2 | Data 📊

The `goal` of this competition is to `predict the function of a set of proteins`. We will `develop a model trained` on the `amino-acid sequences` of the `proteins and on other data`. Our work `will help researchers` better `understand the function of proteins`, which is `important for discovering` `how cells, tissues, and organs work`.


In [1]:
import pandas as pd 
import re
from Bio import SeqIO

The $Training$ $Set$ contains all `proteins with annotated terms` that have been validated by 
* $Experimental$
* $High-Throughput$ $Evidence$
* [$Traceable$ $Author$ $Statement$](https://wiki.geneontology.org/index.php/Traceable_Author_Statement_(TAS)#:~:text=The%20TAS%20evidence%20code%20covers,annotations%20come%20from%20review%20articles.)
* [$Inferred$ $by$ $Curator$ $(IC)$](https://wiki.geneontology.org/Inferred_by_Curator_(IC)) 

**Any other sources of Data are allowed**

### 2.1.1.1 | Go-Basic.obo

The $Ontology$ data is in the `file go-basic.obo`. This file is in $OBO$ `Biology-Oriented Language`. The nodes in `the graph are indexed` by the `term name`
```
subontology_roots = {'BPO':'GO:0008150',
                     'CCO':'GO:0005575',
                     'MFO':'GO:0003674'}
```

In [2]:
with open('/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo') as file :
    
    content = file.read()
    stanzas =  re.findall(r'\[Term\][\s\S]*?(?=\n\[|$)' , content)
    
print(stanzas[0])

[Term]
id: GO:0000001
name: mitochondrion inheritance
namespace: biological_process
def: "The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton." [GOC:mcc, PMID:10873824, PMID:11389764]
synonym: "mitochondrial inheritance" EXACT []
is_a: GO:0048308 ! organelle inheritance
is_a: GO:0048311 ! mitochondrion distribution



### 2.1.1.2 | Training Sequences.fasta

This file contains only `sequences` for `proteins` with `annotations` in the dataset `labeled proteins`.

This files are in `FASTA` format. 

This file contains . To obtain the full set of protein sequences for unlabeled proteins, the Swiss-Prot and TrEMBL databases can be found here.

In [3]:
fasta_seq = SeqIO.parse(open("/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta") ,'fasta')

fasta_seq

### 2.1.1.3 | Train Terms.tsv

This file contains the list of annotated terms `ground truth` for the proteins in `train_sequences.fasta`. 

In [4]:
pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv" , sep = "\t")

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO
...,...,...,...
5363858,X5L565,GO:0050649,MFO
5363859,X5L565,GO:0016491,MFO
5363860,X5M5N0,GO:0005515,MFO
5363861,X5M5N0,GO:0005488,MFO


The first column indicates the `protein's UniProt accession ID`, the second is the `GO term ID`, and the third indicates in which `ontology the term appears`.

### 2.1.1.4 | Train Taxonomy.tsv

This file contains the list of `proteins and the species to which they belong`, represented by a `taxonomic identifier` `taxon ID` number.

In [5]:
pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv" , sep = "\t")

,EntryID,taxonomyID
0,Q8IXT2,9606
1,Q04418,559292
2,A8DYA3,7227
3,Q9UUI3,284812
4,Q57ZS4,185431
...,...,...
142241,Q5TD07,9606
142242,A8BB17,7955
142243,A0A2R8QBB1,7955
142244,P0CT72,284812


### 2.1.1.5 | IA.txt

IA.txt contains the information accretion (weights) for each GO term. These weights are used to compute weighted precision and recall, as described in the Evaluation section. 

## 2.1.2 | Test Set

The $Test$ $Set$ is `unknown at the beginning` of the competition. It will contain `protein sequences` `their functions` from the `test superset` that `gained experimental annotations` between the `submission-deadline` and the `time of evaluation`.

# 3 | PyTorch DataLoader ⚙️

In [6]:
import numpy as np
from transformers import BertModel, BertTokenizer
import torch
from torch.utils.data import Dataset

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

First we will make a simple class...

In [7]:
class Pytorch_Dataset(Dataset):pass

Now we will load the `Embedments` for the data 

Here we will be using the combination of `T5`/`Prot-BERT`

Embeds are like representation of a non-numercial elemenet to a list of numerical elemenet. 

In [8]:
class Pytorch_Dataset(Dataset):
    
    def __init__(self):
        
        super(Pytorch_Dataset).__init__()
        
        self.embeds = np.load("/kaggle/input/t5embeds/test_embeds.npy")

In case you wanna see, what these `embeds`looks like, open the below hidden cells

At this point we will only consider the first $10,000$ values

In [9]:
embeds = np.load("/kaggle/input/cafa-sample-embeddings/Embeddings/T5-Prot-BERT BFD/Train/Embeds.npy")

embeds , embeds.shape

(array([[-0.05582625, -0.04506441, -0.2390366 , ...,  0.14752068,
          0.19537698,  0.01960445],
        [-0.05582625, -0.04506442, -0.2390366 , ...,  0.14752068,
          0.19537698,  0.01960445],
        [-0.05582625, -0.0450644 , -0.2390366 , ...,  0.14752068,
          0.19537698,  0.01960445],
        ...,
        [-0.05582625, -0.04506441, -0.2390366 , ...,  0.14752068,
          0.19537698,  0.01960445],
        [-0.05582625, -0.04506441, -0.2390366 , ...,  0.14752068,
          0.19537698,  0.01960445],
        [-0.05582625, -0.04506441, -0.2390366 , ...,  0.14752068,
          0.19537698,  0.01960445]], dtype=float32),
 (9999, 1024))

For the `training` we need the `targets`, which we will take from the `train_targets_top500.npy`.

In [10]:
class Pytorch_Dataset(Dataset):
    
    def __init__(self , datatype):
        
        super(Pytorch_Dataset).__init__()
        
        self.datatype  = datatype
        
        self.embeds = np.load("/kaggle/input/cafa-sample-embeddings/Embeddings/T5-Prot-BERT BFD/Train/Embeds.npy")
        
        if self.datatype == "train" : 
            
            self.targets = np.load("/kaggle/input/train-targets-top500/train_targets_top500.npy")[:self.embeds.shape[0]]

These are our targets

In [11]:
targets = np.load("/kaggle/input/train-targets-top500/train_targets_top500.npy")[:embeds.shape[0]]

targets

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Now we will just apply some `Getters` in the class 

In [12]:
class Pytorch_Dataset(Dataset):

    def __init__(self , datatype ):
        super(Pytorch_Dataset).__init__()

        self.datatype = datatype
        self.embeds = np.load("/kaggle/input/cafa-sample-embeddings/Embeddings/T5-Prot-BERT BFD/Train/Embeds.npy")
        
        if datatype == "train":

            self.targets = np.load("/kaggle/input/train-targets-top500/train_targets_top500.npy")[:self.embeds.shape[0]]
            
    def __len__(self):return self.targets.shape[0]

    def __getitem__(self , index):

        r_embed = torch.tensor(self.embeds[index] , dtype = torch.float32)

        if self.datatype == "train":

            r_targets = torch.tensor(self.targets[index], dtype = torch.float32)

            return r_embed, r_targets

        return r_embed

In [13]:
train_data = Pytorch_Dataset(datatype = "train")

In [14]:
train_dataloader = torch.utils.data.DataLoader(train_data , batch_size = 128 , shuffle = True)

# 4 | Model Setup 🤖️ 

Now we will train Simple models, and see the results

# 4.1 | Multi Layer Perceptron 🧠

A `MultiLayerPerceptron` is a simple model, which consists of just few perceptrons/nodes/neurons connected with each other. 

As we have choosen for the `top-500` targets, our last layer of the network will contain 500 perceptrons, 

In [15]:
class MultiLayerPerceptron(torch.nn.Module):

    def __init__(self):
      
        super(MultiLayerPerceptron, self).__init__()

        # l_1 = Linear Layer 1 
        # a_1 = Activation Layer 1

        self.l_1 = torch.nn.Linear(1024 , 1000)
        self.a_1 = torch.nn.ReLU()

        self.l_2 = torch.nn.Linear(1000 , 900)
        self.a_2 = torch.nn.ReLU()
        
        self.l_3 = torch.nn.Linear(900 , 800)
        self.a_3 = torch.nn.ReLU()
        
        self.l_4 = torch.nn.Linear(800 , 700)
        self.a_4 = torch.nn.ReLU()
        
        self.l_5 = torch.nn.Linear(700 , 600)
        self.a_5 = torch.nn.ReLU()
        
        self.l_6 = torch.nn.Linear(600 , 500)
    
    def forward(self, x):
        
        x = self.l_1(x)
        x = self.a_1(x)

        x = self.l_2(x)
        x = self.a_2(x)
        
        x = self.l_3(x)
        x = self.a_3(x)
        
        x = self.l_4(x)
        x = self.a_4(x)
        
        x = self.l_5(x)
        x = self.a_5(x)
        
        x = self.l_6(x)
        
        return x

I cannot use `Kaggle GPUs` for some reason (I dont know why), so I will be commenting the `cuda` parts 

In [16]:
MLP = MultiLayerPerceptron()

# MLP = MultiLayerPerceptron().to("cuda")

# 4.2 | Single Layer Perceptron 🧠

A $Single$ $Layer$ $Perceptron$ is a `Neural Network Architechture` that consists of $1$ `Hidden Layer`. According to the `Paper`, we should  be having $1$ `Hidden Layer` with $1,000$ `Perceptrons` in it. 

In [17]:
class SingleLayerPerceptron(torch.nn.Module):

    def __init__(self):
        
        super(SingleLayerPerceptron, self).__init__()

        self.linear1 = torch.nn.Linear(1024, 1012)
        self.activation1 = torch.nn.ReLU()
        
        self.linear2 = torch.nn.Linear(1012, 500)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        
        x = self.linear2(x)
        
        return x

In [18]:
SLP = SingleLayerPerceptron()

# SLP = SingleLayerPerceptron().to("cuda")

## Loss Function 📉

Here we will be using `Categorical Cross Entropy`

In [19]:
CrossEntropy = torch.nn.CrossEntropyLoss()

## Optimizer 💡

Here we will be using `Adam`

In [20]:
optimizer = torch.optim.Adam(MLP.parameters(), lr = 0.0005)

# 5 | Training Loop 🔁 

In [21]:
from IPython.display import IFrame

# 5.1 | Multi Layer Perceptron Training 🔁

Now we will start the training loop 

I dont the exact reason, but everytime I try to access `GPU` for some training in `Kaggle`. `CUDA goes out of memory`. Thus I have trained the model on `Colab` and will imported the results to `Wandb`.

```
los = []

for epochs in (range(5)):

    losses = []

    for x , y in tqdm.tqdm(train_dataloader):
        
        x = torch.tensor(x , dtype = torch.float32)
        y = torch.tensor(x , dtype = troch.float32)
        
        
#         x = torch.tensor(x , dtype = torch.float32).to("cuda")
#         y = torch.tensor(y , dtype = torch.float32).to("cuda")

        optimizer.zero_grad()
        preds = MLP(x)

        loss = CrossEntropy(preds, y)
        losses.append(loss)

    los.append(losses)
```

In [22]:
IFrame("https://wandb.ai//ayushsinghal659/CAFA%7CMLP/reports/CAFA-MLP--Vmlldzo0Nzk1NjQx" , 1300 , 400)

As we can see we did not get good results, but we will try to omprove our results, by improving the model and by adding new one 

# 5.2 | Single Layer Perceptron Training 🔁

Now we will train our `Single Layer Perceptron`

```
# torch.cuda.empty_cache()

los = []

for epochs in (range(4)):

    losses = []
    
    # torch.cuda.empty_cache()
    
    for x , y in tqdm.tqdm(train_dataloader , total = 10000):

        # torch.cuda.empty_cache()

        x = torch.tensor(x , dtype = torch.float32)
        y = torch.tensor(y , dtype = torch.float32)
        
        # x = torch.tensor(x , dtype = torch.float32).to("cuda")
        # y = torch.tensor(y , dtype = torch.float32).to("cuda")

        optimizer.zero_grad()
        preds = SLP(x)

        loss = CrossEntropy(preds, y)
        losses.append(loss)

    los.append(losses)
```

In [23]:
IFrame("https://wandb.ai//ayushsinghal659/uncategorized/reports/CAFA-TEMPROT--Vmlldzo0ODIxNzI3" , 1300 , 400)

# 6 | TO DO LIST 📄

```
TO DO 1 : VISUALIZE THE DATA

TO DO 2 : TRAIN A MODEL

TO DO 3 : TRY DIFFERENT MODELS

TO DO 4 : ADD WANDB SUPPORT

TO DO 5 : ADD TENSORFLOW DATA LOADER

TO DO 6 : TRAIN A TF MODEL

TO DO 7 : IMPROVE RESULTS

TO DO 8 : DECREASE TRAINING TIME

TO DO 9 : DANCE 
```

# 7 | Ending 🏁

**THAT'S IT FOR TODAY GUYS**

**WE WILL GO DEEPER INTO THE DATA IN THE UPCOMING VERSIONS**

**PLEASE COMMENT YOUR THOUGHTS, HIHGLY APPRICIATED**

**DONT FORGET TO MAKE AN UPVOTE, IF YOU LIKED MY WORK $:)$**

<img src = "https://i.imgflip.com/19aadg.jpg">

**PEACE OUT $:)$**